## What is Chunking and Why Does it Matter?

Chunking is the process of breaking down documents into smaller pieces that can be efficiently processed by language models and retrieval systems. The way you chunk your documents directly impacts:

- **Retrieval Precision**: How accurately your system can find relevant information
- **Context Preservation**: How much surrounding information is maintained
- **Token Economy**: How efficiently you use your LLM's context window
- **Storage Requirements**: How much vector storage you need

Let's explore different chunking strategies and their impact on document retrieval.

In [ ]:
from llama_index.core.schema import Document
import textwrap

# Sample document text
sample_text = """
# Introduction to Vector Databases

Vector databases are specialized database systems designed to store and query vector embeddings efficiently.
Unlike traditional databases optimized for exact matches, vector databases excel at similarity searches.

## Key Advantages

Vector databases offer several advantages for AI applications:
- Efficient similarity search using algorithms like HNSW and IVF
- Support for high-dimensional vector data
- Optimized for retrieval-augmented generation (RAG) applications

## Common Operations

The most common operations in vector databases include:
1. Adding vectors with associated metadata
2. Searching for similar vectors using distance metrics
3. Filtering results based on metadata
4. Building and optimizing indexes for faster retrieval

# Performance Considerations

When working with vector databases at scale, consider:
- Index construction time vs. query performance
- Memory usage vs. search accuracy
- Batch processing for efficient vector insertion
"""

# Create a Document
document = Document(text=sample_text)

# Let's print the original document to understand its structure
print("Original Document:")
print(textwrap.fill(document.text[:500], 100))
print("...\n")

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

# Sentence-based chunking
sentence_splitter = SentenceSplitter(
    chunk_size=40,  # Target chunk size (in characters)
    chunk_overlap=10  # Overlap between chunks (in characters)
)

sentence_nodes = sentence_splitter.get_nodes_from_documents([document])

print(f"Sentence Splitting created {len(sentence_nodes)} chunks\n")
print("Example chunks:")
for i in range(min(3, len(sentence_nodes))):
    print(f"\nChunk {i}:")
    print(textwrap.fill(sentence_nodes[i].text[:300], 100))
    print(f"Character length: {len(sentence_nodes[i].text)}")

In [ ]:
from llama_index.core.node_parser import TokenTextSplitter

# Token-based chunking
token_splitter = TokenTextSplitter(
    chunk_size=40,  # Target chunk size (in tokens)
    chunk_overlap=10  # Overlap between chunks (in tokens)
)

token_nodes = token_splitter.get_nodes_from_documents([document])

print(f"Token Splitting created {len(token_nodes)} chunks\n")
print("Example chunks:")
for i in range(min(3, len(token_nodes))):
    print(f"\nChunk {i}:")
    print(textwrap.fill(token_nodes[i].text[:300], 100))
    print(f"Character length: {len(token_nodes[i].text)}")

In [ ]:
from llama_index.core.node_parser import HierarchicalNodeParser

# Hierarchical chunking
hierarchical_splitter = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[70, 60, 40]  # Multi-level chunking
)

hierarchical_nodes = hierarchical_splitter.get_nodes_from_documents([document])

print(f"Hierarchical Splitting created {len(hierarchical_nodes)} chunks\n")
print("Example chunks:")
for i in range(min(3, len(hierarchical_nodes))):
    print(f"\nChunk {i}:")
    print(textwrap.fill(hierarchical_nodes[i].text[:300], 100))
    print(f"Character length: {len(hierarchical_nodes[i].text)}")

In [ ]:
from llama_index.core.node_parser import MarkdownNodeParser

# Structure-aware chunking for Markdown
markdown_splitter = MarkdownNodeParser()

markdown_nodes = markdown_splitter.get_nodes_from_documents([document])

print(f"\nMarkdown-aware Splitting created {len(markdown_nodes)} chunks\n")
print("Example chunks with their headings:")
for i in range(min(3, len(markdown_nodes))):
    heading = markdown_nodes[i].metadata.get('heading', 'No heading')
    print(f"\nChunk {i} (Heading: {heading}):")
    print(textwrap.fill(markdown_nodes[i].text[:200], 100))

# Let's try the different strategies!

In [7]:
# Special lib needed for local embeddings
%pip install llama-index-embeddings-huggingface

Defaulting to user installation because normal site-packages is not writeable
  Using cached llama_index_embeddings_huggingface-0.5.2-py3-none-any.whl.metadata (767 bytes)
  Using cached huggingface_hub-0.29.3-py3-none-any.whl.metadata (13 kB)
  Using cached sentence_transformers-4.0.1-py3-none-any.whl.metadata (13 kB)
  Using cached transformers-4.50.3-py3-none-any.whl.metadata (39 kB)
  Using cached torch-2.6.0-cp312-cp312-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached llama_index_embeddings_huggingface-0.5.2-py3-none-any.whl (8.9 kB)
Using cached huggingface_hub-0.29.3-py3-none-any.whl (468 kB)
Using cached sentence_transformers-4.0.1-py3-none-any.whl (340 kB)
Using cached torch-2.6.0

In [9]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Create a local embedding model
local_embed_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")

# Create vector stores with different chunking strategies (using local embeddings)
sentence_index = VectorStoreIndex(
    sentence_nodes, embed_model=local_embed_model)
token_index = VectorStoreIndex(token_nodes, embed_model=local_embed_model)
markdown_index = VectorStoreIndex(
    markdown_nodes, embed_model=local_embed_model)

# Query to test retrieval
query = "What are the top operations in vector databases?"

# Get retrieval results
sentence_results = sentence_index.as_retriever().retrieve(query)
token_results = token_index.as_retriever().retrieve(query)
markdown_results = markdown_index.as_retriever().retrieve(query)

# Compare top results
print("\nRETRIEVAL COMPARISON\n")
print("Sentence chunking result:")
print(textwrap.fill(sentence_results[0].node.text[:300], 100))

print("\nToken chunking result:")
print(textwrap.fill(token_results[0].node.text[:300], 100))

print("\nMarkdown-aware chunking result:")
print(textwrap.fill(markdown_results[0].node.text[:300], 100))



RETRIEVAL COMPARISON

Sentence chunking result:
common operations in vector databases include: 1. Adding vectors with associated metadata 2.
Searching for similar vectors using distance metrics 3. Filtering results based on metadata 4.

Token chunking result:
generation (RAG) applications  ## Common Operations  The most common operations in vector databases
include: 1. Adding vectors with associated metadata 2. Searching for similar vectors using distance

Markdown-aware chunking result:
## Common Operations  The most common operations in vector databases include: 1. Adding vectors with
associated metadata 2. Searching for similar vectors using distance metrics 3. Filtering results
based on metadata 4. Building and optimizing indexes for faster retrieval


## Conclusions

1. **Chunk Size Tradeoffs**: Smaller chunks allow for more precise retrieval but may lose context. Larger chunks preserve more context but might introduce noise and use more tokens from your LLM's context window.

2. **Overlap Between Chunks**: dding overlap ensures that sentences or ideas that cross chunk boundaries aren't lost, but increases storage requirements and can create duplicate information in retrieval.

3. **Structure Awareness**: Domain-specific chunking that understands document structure (like our markdown example) typically produces better results but requires more specialized processing.